<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np

from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

import csv
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score

from nltk.stem import WordNetLemmatizer
m = WordNetLemmatizer() 
import re
from tqdm import notebook
notebook.tqdm.pandas()

In [ ]:
random_state = 12345

In [2]:
data = pd.read_csv('toxic_comments.csv')

In [3]:
display(data.head())
display(data.info())

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

In [4]:
display(data['toxic'].value_counts())
data['toxic'].value_counts(normalize=True).map('{:.2%}'.format)

toxic
0    143106
1     16186
Name: count, dtype: int64

toxic
0    89.84%
1    10.16%
Name: proportion, dtype: object

Целевой признак с очень большим дисбалансом.

### Подготовка данных

In [5]:
def text_tokenize(text):
    tokenized = nltk.word_tokenize(text)
    joined = ' '.join(tokenized)
    text_only = re.sub(r'[^a-zA-Z]', ' ', joined)
    final = ' '.join(text_only.split())
    x =  "".join(m.lemmatize(final))
    return x

In [6]:
display(data['text'][0])

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [7]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /Users/vs/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/vs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
notebook.tqdm.pandas() 
data['text_final'] = data['text'].progress_apply(text_tokenize)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [9]:
display(data['text_final'][0])

'Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They were n t vandalisms just closure on some GAs after I voted at New York Dolls FAC And please do n t remove the template from the talk page since I m retired now'

In [10]:
features = data['text_final']
target = data['toxic']

In [11]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=12345, 
                                                                            stratify=target)
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(119469,)
(119469,)
(39823,)
(39823,)


In [12]:
print('Распределение классов в обучающей выборке:')
print(target_train.value_counts()[0] / target_train.value_counts().sum())
print(target_train.value_counts()[1] / target_train.value_counts().sum())
print()
print('Распределение классов в тестовой выборке:')
print(target_test.value_counts()[0] / target_test.value_counts().sum())
print(target_test.value_counts()[1] / target_test.value_counts().sum())

Распределение классов в обучающей выборке:
0.8983836811222995
0.10161631887770049

Распределение классов в тестовой выборке:
0.8984004218667604
0.10159957813323958


In [13]:
corpus_train = features_train.values.astype('U')
corpus_test = features_test.values.astype('U')

In [18]:
nltk.download('stopwords')
stopwords = list(set(nltk_stopwords.words('english')))
display(len(stopwords))

[nltk_data] Downloading package stopwords to /Users/vs/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [19]:
tf_idf = TfidfVectorizer(stop_words=stopwords)

In [20]:
features_train_data_tf_idf = tf_idf.fit_transform(corpus_train)
features_test_data_tf_idf = tf_idf.transform(corpus_test)

In [21]:
print('TF-IDF train')
print(features_train_data_tf_idf.shape)
print(features_train_data_tf_idf[:5].toarray())
print()
print('TF-IDF test')
print(features_test_data_tf_idf.shape)
print(features_test_data_tf_idf[:5].toarray())

TF-IDF train
(119469, 142615)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

TF-IDF test
(39823, 142615)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Вывод:

* Токенизировали и лемматизировали столбец text и добавили данные в столбец text_final
* Провели оценку важности слов определением величины TF-IDF
* Проверили данные после векторизации

## Обучение

## Обучим следующие модели:

* LogisticRegression
* RandomForestClassifier
* DecisionTreeClassifier

In [36]:
model = LogisticRegression(max_iter=500, random_state=random_state, class_weight='balanced')
model.fit(features_train_data_tf_idf, target_train)
predictions = model.predict(features_test_data_tf_idf)
    
test_f1_score = f1_score(target_test, predictions)

In [37]:
results = pd.DataFrame({'Preprocessing model' : ['TF-IDF'],
                        'Learning model' : ['LogisticRegression'], 
                        'Test f1 score' : round(test_f1_score, 2)})

In [38]:
model = RandomForestClassifier(random_state=random_state, class_weight = 'balanced')
model.fit(features_train_data_tf_idf, target_train)
predictions = model.predict(features_test_data_tf_idf)
    
test_f1_score = f1_score(target_test, predictions)

In [39]:
results2 = pd.DataFrame({'Preprocessing model' : ['TF-IDF'],
                        'Learning model' : ['LogisticRegression'], 
                        'Test f1 score' : round(test_f1_score, 2)})

In [40]:
model = DecisionTreeClassifier(random_state=random_state, class_weight = 'balanced')
model.fit(features_train_data_tf_idf, target_train)
predictions = model.predict(features_test_data_tf_idf)
    
test_f1_score = f1_score(target_test, predictions)

In [41]:
results3 = pd.DataFrame({'Preprocessing model' : ['TF-IDF'],
                        'Learning model' : ['LogisticRegression'], 
                        'Test f1 score' : round(test_f1_score, 2)})

In [42]:
results = pd.concat([results, results2], ignore_index=True)
results = pd.concat([results, results3], ignore_index=True)

display(results)

,Preprocessing model,Learning model,Test f1 score
0,TF-IDF,LogisticRegression,0.76
1,TF-IDF,LogisticRegression,0.62
2,TF-IDF,LogisticRegression,0.63


## Выводы

* Данные загружены, состоят из 159571 строк и 2 столбцов
* Столбец toxic является целевым признаком с выраженым дисбалансом классов
* Токенизировали и лемматизировали столбец text и добавим данные в столбец text_final
* Провели оценку важности слов определением величины TF-IDF
* Проверили данные после векторизации
* Обучили следующие модели:
    * LogisticRegression
    * RandomForestClassifier
    * DecisionTreeClassifier
* Лучший результат метрики F1-score 0.76 показала модель LogisticRegression
* Другие модели не прошли по условию F1-score не меньше 0.75 и время обучения и предсказания намного выше чем у модели LogisticRegression
* Модель LogisticRegression можно рекомендовать для использования в качестве инструмента, который будет искать токсичные комментарии и отправлять их на модерацию.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны